In [1]:
from pylab import *
import pandas as pd


#%qtconsole --style monokai
%matplotlib inline
plt.style.use('classic')
plt.rc("figure",facecolor="w",figsize=(6,4))
plt.rc("font",size=10)
plt.rc("savefig",dpi=300)

# 0. Load MC Events

In [27]:
df0  = pd.read_csv("data/25_3/ttbar_mumu.csv")
df1  = pd.read_csv("data/25_3/ttbar_mutau_mumu.csv")
df2  = pd.read_csv("data/25_3/ttbar_tautau_mumu.csv")



removecollist = ['lepton1_charge','lepton1_mother','lepton2_charge','lepton2_mother',
                 'lepton1_phi','lepton2_phi','dilepton_phi',
                 'jet1_phi',"jet2_phi",'dijet_phi','lb1_phi','lb2_phi','met','mpt','metphi'
                ]
df0.drop(removecollist, axis=1, inplace=True)
df1.drop(removecollist, axis=1, inplace=True)
df2.drop(removecollist, axis=1, inplace=True)
df0 = df0.reindex()
df1 = df1.reindex()
df2 = df2.reindex()


#df1 = df1.sample(frac=0.1739*2)
#df2 = df2.sample(frac=0.1739**2)

df0["label"]=0*np.ones(len(df0),int)
df1["label"]=1*np.ones(len(df1),int)
df2["label"]=1*np.ones(len(df2),int)

df_train = pd.concat([df0,df1,df2],ignore_index=True)

# 1. Train

In [4]:
from utility_DNN import *
from IPython.display import clear_output
net = Net(40,128,128,2).cuda()
#net = torch.load("checkpointDNN_temp.pt")

In [29]:
BATCHSIZE   = 500
trainset    = MyDataset(df_train.as_matrix())
trainloader = DataLoader(trainset, batch_size=BATCHSIZE,shuffle=True, num_workers=4)

criterion   = nn.CrossEntropyLoss()
optimizer   = optim.SGD(net.parameters(), lr=0.000001, momentum=0.9)
net = net.cuda()

In [ ]:
net = net.cuda()
for epoch in range(20):
    running_loss = 0.0
    for i, onebatch in enumerate(trainloader,0):
        # get the inputs
        inputs = Variable(onebatch["feature"]).cuda()
        labels = Variable(onebatch["label"]).cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.data[0]
        
    clear_output(wait=True)
    print('[%d] loss: %.5f' %(epoch + 1, running_loss))
print('Finished Training')

[16] loss: 107.89168


In [24]:
classes = ["Notau","Yestau"]
class_correct = list(0. for i in range(2))
class_total   = list(0. for i in range(2))
net           = net.cpu()
for dataiter in trainloader:
    #dataiter = iter(trainloader).next()
    thisbatch_inputs   = dataiter["feature"]
    thisbatch_labels   = dataiter["label"]
    thisbatch_outputs  = net(Variable(thisbatch_inputs))
    thisbatch_predicts = torch.max(thisbatch_outputs.data, 1)[1] # return likelihood, predict
    c = (thisbatch_predicts == thisbatch_labels).squeeze()
    for i in range(BATCHSIZE):
        if len(c) == BATCHSIZE:
            class_correct[thisbatch_labels[i]] += c[i]
            class_total[thisbatch_labels[i]]   += 1
            
for i in range(2):
    print ("Accuracy of {:12s} : {:3.1f}%".format(classes[i],100*class_correct[i]/class_total[i]))

Accuracy of Notau        : 76.1%
Accuracy of Yestau       : 74.9%


In [26]:
torch.save(net.cpu(),"checkpointDNN.pt")

In [ ]:
hist(df2.met,histtype="step",color="b",normed=True,bins=np.arange(0,2500,50))
hist(df1.met,histtype="step",color="g",normed=True,bins=np.arange(0,2500,50))
hist(df0.met,histtype="step",color="r",normed=True,bins=np.arange(0,2500,50))